<a href="https://colab.research.google.com/github/francesco-mollica/word2vec-entity_embeddings/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
base_dir_models = root_dir + "tesi_magistrale/datasets/models_w2v/"

Mounted at /content/drive/


In [ ]:
class MySentences(object):
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
            for line in open(base_dir + self.filename):
                yield line.split()

In [ ]:
import numpy as np
from gensim.models import Word2Vec
def restrict_w2v(w2v, restricted_word_set):
    new_vectors = []
    new_vocab = {}
    new_index2entity = []
    new_vectors_norm = []

    for i in range(len(w2v.wv.vocab)):
        word = w2v.wv.index2entity[i]
        vec = w2v.wv.vectors[i]
        vocab = w2v.wv.vocab[word]
        vec_norm = w2v.wv.vectors_norm[i]
        if word in restricted_word_set:
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            new_vectors_norm.append(vec_norm)

    w2v.wv.vocab = new_vocab
    w2v.wv.vectors = np.array(new_vectors)
    w2v.wv.index2entity = np.array(new_index2entity)
    w2v.wv.index2word = np.array(new_index2entity)
    w2v.wv.vectors_norm = np.array(new_vectors_norm)

In [ ]:
names = {1:"balanced_ontonotes", 2:"ontonotes", 3:"bbn", 4:"figer", 5:"choi"}

In [ ]:
for value in names.items():
  with open(base_dir + "restricted_list_" + value + ".pkl", 'rb') as f:
    restricted_word_list = pickle.load(f)
  
  sentences = MySentences("corpus_data_entire_" + value + ".txt")
  w2v_model = Word2Vec(iter=1, min_count=1)
  w2v_model.build_vocab(sentences)
  w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
  # execute one time a most_similar otherwise vectors_norm in restrict_w2v should be equal to None
  w2v_model.wv.most_similar(positive=restricted_word_list[0])
  # restrict model
  restrict_w2v(w2v_model, restrict_word_list)
  # save
  w2v_model.save(base_dir_models + "word2vec_" + value + ".model")

  del restricted_word_list
  del sentences
  del w2v_model